In [1]:
from models import tokenize

d:\anaconda\envs\pt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = tokenize(["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"])
text

tensor([[ 101, 3345, 2225, 7991,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 101, 1975, 6032, 4905, 2094,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 101, 2207, 4125, 7987,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,   

In [3]:
import torch
from PIL import Image
from models import load_from_name, available_models
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device:{device}')
model, preprocess = load_from_name("ViT-B-16", device=device, download_root='./')


device:cuda
Loading vision model config from e:\explore\CLI2P\models\model_configs\ViT-B-16.json
Loading text model config from e:\explore\CLI2P\models\model_configs\RoBERTa-wwm-ext-base-chinese.json
Model info {'embed_dim': 512, 'image_resolution': 224, 'vision_layers': 12, 'vision_width': 768, 'vision_patch_size': 16, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 12, 'text_type_vocab_size': 2}


In [4]:
model.eval()
image = preprocess(Image.open("examples/pokemon.jpeg")).unsqueeze(0).to(device)   # .shape=torch.Size([1, 3, 224, 224])
# text = tokenize(["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"]).to(device)
text = tokenize(["杰尼龟"]).to(device) # .shape=[1,52]  ===> [ [101(开始标志), 3345('杰'的索引), 2225('尼'的索引), 7991('龟'的索引) ， 0 ，0， 0，... 0  ] ]  ---> 所谓索引就是该字符在 models/vocab.txt 中的行数

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    # 对特征进行归一化，请使用归一化后的图文特征用于下游任务
    image_features /= image_features.norm(dim=-1, keepdim=True)  # 
    text_features /= text_features.norm(dim=-1, keepdim=True)    
    print(f'image_features.shape:{image_features.shape}')  # image_features.shape:torch.Size([1, 512])   可见 图像编码 与 文本编码 最后都是编码成为 512 
    print(f'text_features.shape:{text_features.shape}')    # text_features.shape:torch.Size([1, 512])
    logits_per_image, logits_per_text = model.get_similarity(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]

d:\anaconda\envs\pt\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


image_features.shape:torch.Size([1, 512])
text_features.shape:torch.Size([1, 512])
Label probs: [[1.]]


In [5]:
image_features

tensor([[ 9.5581e-02, -3.7445e-02,  1.6556e-02,  7.1297e-03, -2.9434e-02,
          2.7298e-02,  6.1874e-03,  3.7048e-02,  1.2253e-02, -9.0637e-03,
          2.0008e-03, -2.7954e-02,  2.8717e-02, -7.5188e-03, -4.3983e-03,
         -2.3956e-03,  1.3840e-02, -1.4870e-02,  2.2354e-02, -4.2415e-04,
         -3.0045e-02,  3.7415e-02,  4.5227e-02, -1.2169e-02,  8.6975e-03,
          3.2440e-02, -5.2246e-02,  6.8054e-02,  2.8107e-02,  2.6657e-02,
          5.0125e-03,  3.0945e-02, -2.6260e-02,  7.6599e-02,  1.9211e-02,
         -1.5137e-02,  4.3671e-02,  1.9699e-02,  9.2407e-02,  6.0883e-02,
         -1.1253e-02, -1.7456e-02,  2.9648e-02, -1.9197e-03, -3.7556e-03,
          7.4501e-03,  1.4351e-02,  1.6083e-02,  1.4315e-03, -2.1835e-02,
          3.0807e-02,  4.0398e-03,  1.7715e-02, -3.9246e-02,  1.3855e-02,
         -3.5187e-02,  2.9419e-02,  1.8738e-02, -9.1934e-03, -8.6670e-03,
          2.5757e-02,  1.4954e-02,  6.0797e-04, -3.7994e-02,  2.5902e-03,
         -2.3743e-02,  2.4399e-02,  7.

In [6]:
text_features

tensor([[ 1.1743e-01, -1.6266e-02,  3.9005e-03, -2.1378e-02,  3.2845e-03,
          3.6865e-02,  5.6641e-02,  5.1880e-03,  5.6641e-02,  5.8403e-03,
         -4.9561e-02, -7.6660e-02, -2.1606e-02, -1.4366e-02, -2.5574e-02,
          2.7725e-02, -1.0590e-02, -6.5552e-02,  9.5032e-02,  4.6204e-02,
         -4.4937e-03,  5.7770e-02, -3.5858e-02,  1.0719e-02,  1.1894e-02,
         -3.0746e-02,  2.9526e-03,  9.8114e-03, -9.5749e-03,  2.2476e-02,
          2.2526e-03,  5.4321e-03,  8.4305e-03,  8.1360e-02,  2.7313e-02,
          3.1097e-02, -3.9024e-03, -5.0598e-02,  4.1389e-03, -3.0731e-02,
          2.1164e-02, -2.0462e-02, -2.4460e-02,  1.8967e-02,  5.9471e-03,
         -2.7981e-03,  5.5969e-02,  3.8452e-02,  2.5146e-02, -7.7667e-03,
          2.1515e-02,  4.7058e-02, -5.0110e-02, -1.2871e-02, -4.9072e-02,
         -3.7415e-02, -8.4595e-02,  1.6464e-02,  1.7944e-02, -2.3918e-03,
          1.4763e-02, -3.1090e-03, -9.3994e-02, -3.1708e-02, -2.3975e-03,
         -6.1188e-02, -4.2633e-02,  3.